# Comprehensive API Test Notebook

This notebook exercises the hierarchy, question, search, export, and PYQ endpoints exposed by the ITS question service. Configure the base URL and optional auth token in the configuration cell before running the suite.

Sections:
1. Configuration & helpers
2. Hierarchy scaffolding (creates seed data)
3. Hierarchy & analytics endpoints
4. Question workflows
5. Search, item-bank, and exports
6. PYQ services
7. Summary report

Each run uses timestamped labels to avoid collisions with existing records.


In [6]:
import os
import json
import uuid
from datetime import datetime

import requests
import pandas as pd
from IPython.display import display, JSON

BASE_URL = os.getenv("API_BASE_URL", "http://localhost:5200/api").rstrip("/")
AUTH_TOKEN = os.getenv("API_AUTH_TOKEN")

session = requests.Session()
session.headers.update({"Content-Type": "application/json"})
if AUTH_TOKEN:
    session.headers["Authorization"] = f"Bearer {AUTH_TOKEN}"

print(f"Base URL -> {BASE_URL}")
print("Auth token detected." if AUTH_TOKEN else "Auth token not provided (set API_AUTH_TOKEN to add one).")

RUN_LABEL = datetime.utcnow().strftime("%Y%m%d-%H%M%S")
print(f"Run label: {RUN_LABEL}")


Base URL -> http://localhost:5200/api
Auth token not provided (set API_AUTH_TOKEN to add one).
Run label: 20251003-145714


In [7]:
from typing import Any, Callable, Iterable, Optional

test_log: list[dict[str, Any]] = []
state: dict[str, Any] = {}

def build_url(endpoint: str) -> str:
    return f"{BASE_URL}/{endpoint.lstrip('/')}"

def preview(body: Any, max_items: int = 5) -> None:
    if isinstance(body, list):
        subset = body[:max_items]
        if subset and isinstance(subset[0], dict):
            display(pd.DataFrame(subset))
        else:
            display(JSON(subset))
        if len(body) > max_items:
            print(f"... {len(body) - max_items} more items")
    elif isinstance(body, dict):
        display(JSON(body))
    elif body:
        print(body)
    else:
        print("No content")

def remember(key: str, value: Any) -> Any:
    state[key] = value
    return value

def recall(key: str) -> Any:
    if key not in state:
        raise KeyError(f"Missing key in state: {key}")
    return state[key]

def store_resource(key: str) -> Callable[[Any], None]:
    def _inner(data: Any) -> None:
        if isinstance(data, dict):
            remember(key, data)
            resource_id = data.get("id")
            if resource_id:
                remember(f"{key}_id", resource_id)
    return _inner

def run_test(
    name: str,
    method: str,
    endpoint: str,
    *,
    params: Optional[dict[str, Any]] = None,
    json_body: Optional[dict[str, Any]] = None,
    data: Optional[dict[str, Any]] = None,
    files: Optional[dict[str, Any]] = None,
    expect: Iterable[int] | int = 200,
    capture: bool = True,
    store: Optional[Callable[[Any], None]] = None,
    raise_on_error: bool = False,
) -> tuple[Any, requests.Response]:
    url = build_url(endpoint)
    response = session.request(
        method.upper(),
        url,
        params=params,
        json=json_body,
        data=data,
        files=files,
    )
    status = response.status_code
    try:
        body = response.json()
    except ValueError:
        body = response.text

    expected_codes = {expect} if isinstance(expect, int) else set(expect)
    success = status in expected_codes

    row = {
        "name": name,
        "method": method.upper(),
        "endpoint": endpoint,
        "url": url,
        "status": status,
        "expected": sorted(expected_codes),
        "success": success,
        "params": params or {},
    }
    if json_body is not None:
        row["payload"] = json_body

    display(pd.Series({k: (json.dumps(v, indent=2) if isinstance(v, (dict, list)) else v) for k, v in row.items()}))
    preview(body)

    if capture:
        test_log.append({**row, "response": body})

    if store and success:
        store(body)

    if raise_on_error and not success:
        response.raise_for_status()

    return body, response


In [8]:
def unique_label(prefix: str) -> str:
    return f"{prefix} {RUN_LABEL}-{uuid.uuid4().hex[:6]}"

def create_exam() -> None:
    payload = {
        "name": unique_label("API Test Exam"),
        "description": "Seeded by comprehensive API notebook."
    }
    run_test(
        "Create exam",
        "POST",
        "exams",
        json_body=payload,
        expect=201,
        store=store_resource("exam"),
    )

def create_subject() -> None:
    payload = {
        "exam_id": recall("exam_id"),
        "name": unique_label("API Test Subject"),
        "description": "Subject created for automated API coverage."
    }
    run_test(
        "Create subject",
        "POST",
        "subjects",
        json_body=payload,
        expect=201,
        store=store_resource("subject"),
    )

def create_chapter() -> None:
    payload = {
        "subject_id": recall("subject_id"),
        "name": unique_label("API Test Chapter"),
        "description": "Chapter created during API notebook run."
    }
    run_test(
        "Create chapter",
        "POST",
        "chapters",
        json_body=payload,
        expect=201,
        store=store_resource("chapter"),
    )

def create_topic() -> None:
    payload = {
        "chapter_id": recall("chapter_id"),
        "name": unique_label("API Test Topic"),
        "description": "Topic created during API notebook run."
    }
    run_test(
        "Create topic",
        "POST",
        "topics",
        json_body=payload,
        expect=201,
        store=store_resource("topic"),
    )

def create_concept() -> None:
    payload = {
        "topic_id": recall("topic_id"),
        "name": unique_label("API Test Concept"),
        "description": "Concept created during API notebook run."
    }
    run_test(
        "Create concept",
        "POST",
        "concepts",
        json_body=payload,
        expect=201,
        store=store_resource("concept"),
    )

def create_attribute() -> None:
    payload = {
        "name": unique_label("API Attribute"),
        "description": "Attribute created during API notebook run.",
        "concept_id": recall("concept_id")
    }
    run_test(
        "Create attribute",
        "POST",
        "attributes",
        json_body=payload,
        expect=201,
        store=store_resource("attribute"),
    )

def generate_attribute_suggestions() -> None:
    payload = {
        "content": "Which European city is known as the City of Lights?",
        "options": ["Rome", "London", "Paris", "Berlin"],
        "correct_answer": "Paris",
        "exam_id": recall("exam_id"),
        "subject_id": recall("subject_id"),
        "chapter_id": recall("chapter_id"),
        "topic_id": recall("topic_id"),
        "concept_id": recall("concept_id")
    }
    run_test(
        "Generate attribute suggestions",
        "POST",
        "questions/generate-attributes",
        json_body=payload,
        expect=200,
        store=lambda data: remember("attribute_suggestions", data),
    )

def store_question(data: Any) -> None:
    if isinstance(data, dict):
        question = data.get("question")
        if isinstance(question, dict):
            remember("question", question)
            question_id = question.get("id")
            if question_id:
                remember("question_id", question_id)

def create_question() -> None:
    payload = {
        "question": {
            "content": unique_label("What is the capital of France?"),
            "options": ["Paris", "London", "Berlin", "Madrid"],
            "correct_answer": "Paris",
            "exam_id": recall("exam_id"),
            "subject_id": recall("subject_id"),
            "chapter_id": recall("chapter_id"),
            "topic_id": recall("topic_id"),
            "concept_id": recall("concept_id")
        },
        "selected_attributes": [
            {
                "attribute_id": recall("attribute_id"),
                "value": True
            }
        ],
        "create_new_attributes": []
    }
    run_test(
        "Create question (with attributes)",
        "POST",
        "questions/create-with-attributes",
        json_body=payload,
        expect=201,
        store=store_question,
    )

def store_pyq_upload(data: Any) -> None:
    if isinstance(data, dict) and data.get("success"):
        question = data.get("question")
        if isinstance(question, dict):
            remember("pyq_question", question)
            question_id = question.get("id")
            if question_id:
                remember("pyq_question_id", question_id)

def store_pyq_session(data: Any) -> None:
    if isinstance(data, dict) and data.get("success"):
        session_info = data.get("session")
        if isinstance(session_info, dict):
            remember("pyq_session", session_info)
            session_id = session_info.get("id")
            if session_id:
                remember("pyq_session_id", session_id)
            question_ids = session_info.get("question_ids")
            if question_ids:
                remember("pyq_session_question_ids", question_ids)


In [10]:
create_exam()
create_subject()
create_chapter()
create_topic()
create_concept()
create_attribute()
generate_attribute_suggestions()
create_question()

id_snapshot = {k: v for k, v in state.items() if k.endswith("_id")}
display(pd.Series(id_snapshot))


name                                              Create exam
method                                                   POST
endpoint                                                exams
url                           http://localhost:5200/api/exams
status                                                    201
expected                                          [\n  201\n]
success                                                  True
params                                                     {}
payload     {\n  "name": "API Test Exam 20251003-145714-a1...
dtype: object

,created_at,description,id,name,updated_at
0,2025-10-03T15:05:33.149049+00:00,Seeded by comprehensive API notebook.,1879e305-dd3a-4e6b-b9a4-b76d1232015b,API Test Exam 20251003-145714-a11f0e,2025-10-03T15:05:33.149049+00:00


KeyError: 'Missing key in state: exam_id'

In [ ]:
run_test("List exams", "GET", "hierarchy/exams")
run_test("List subjects for exam", "GET", "hierarchy/subjects", params={"exam_id": recall("exam_id")})
run_test("List chapters for subject", "GET", "hierarchy/chapters", params={"subject_id": recall("subject_id")})
run_test("List topics for chapter", "GET", "hierarchy/topics", params={"chapter_id": recall("chapter_id")})
run_test("List concepts for topic", "GET", "hierarchy/concepts", params={"topic_id": recall("topic_id")})
run_test("Get hierarchy children", "GET", f"hierarchy/subject/{recall('subject_id')}/children")
run_test("Get hierarchy chain", "GET", f"hierarchy/concept/{recall('concept_id')}/chain")
run_test("Get hierarchy tree", "GET", "hierarchy/tree")
run_test("Concept question count", "GET", f"hierarchy/concept/{recall('concept_id')}/question-count")
run_test("Subject question count", "GET", f"hierarchy/subject/{recall('subject_id')}/question-count")
run_test("Exam question count", "GET", f"hierarchy/exam/{recall('exam_id')}/question-count")
run_test("Concept stats", "GET", f"hierarchy/concept/{recall('concept_id')}/stats")


In [ ]:
run_test("List attributes for concept", "GET", "hierarchy/attributes", params={"concept_id": recall("concept_id")})
run_test("Concept attributes detail", "GET", f"concept/{recall('concept_id')}/attributes")
run_test("List questions by concept", "GET", "questions", params={"concept_id": recall("concept_id")})
run_test("Get question by id", "GET", f"questions/{recall('question_id')}")
run_test("Questions under concept hierarchy", "GET", f"hierarchy/concept/{recall('concept_id')}/questions")
run_test("Batch get questions", "POST", "questions/batch-get", json_body={"question_ids": [recall("question_id")]})
run_test("Item bank snapshot", "GET", f"item-bank/concept/{recall('concept_id')}")


In [ ]:
run_test("Search hierarchy", "GET", "search/hierarchy", params={"query": "API Test"})
run_test("Search questions", "GET", "search/questions", params={"text_search": "capital", "page_size": 10})
run_test("Export EduCDM snapshot", "GET", f"export/educdm/concept/{recall('concept_id')}")


In [ ]:
pyq_payload = {
    "content": unique_label("Which planet is known as the Red Planet?"),
    "options": ["Mercury", "Venus", "Earth", "Mars"],
    "correct_answer": "Mars",
    "exam_id": recall("exam_id"),
    "subject_id": recall("subject_id"),
    "chapter_id": recall("chapter_id"),
    "topic_id": recall("topic_id"),
    "concept_id": recall("concept_id"),
    "metadata": {
        "year": datetime.utcnow().year - 1,
        "exam_session": "Mock Session",
        "paper_code": f"API-{RUN_LABEL}",
        "question_number": "1",
        "marks_allocated": 1.0,
        "time_allocated": 90,
        "solution": "Mars is called the Red Planet because of its iron oxide rich surface.",
        "source": "API Notebook",
        "tags": ["api-test", RUN_LABEL],
        "difficulty_level": "Medium",
        "question_type": "MCQ"
    },
    "attributes": [
        {
            "id": recall("attribute_id"),
            "value": True
        }
    ]
}
run_test(
    "PYQ upload single",
    "POST",
    "pyq/upload/single",
    json_body=pyq_payload,
    expect=(200, 201),
    store=store_pyq_upload,
)

bulk_payload = {
    "questions": [
        {
            **pyq_payload,
            "content": unique_label("Bulk PYQ question A"),
            "metadata": {
                **pyq_payload["metadata"],
                "question_number": "2"
            }
        },
        {
            **pyq_payload,
            "content": unique_label("Bulk PYQ question B"),
            "metadata": {
                **pyq_payload["metadata"],
                "question_number": "3"
            }
        }
    ]
}
run_test("PYQ upload bulk", "POST", "pyq/upload/bulk", json_body=bulk_payload, expect=200)

run_test("PYQ statistics", "GET", "pyq/statistics")
run_test("PYQ search", "GET", "pyq/search", params={"exam_id": recall("exam_id"), "page_size": 5})
run_test("PYQ filter options", "GET", "pyq/filters/options")
run_test("PYQ template download", "GET", "pyq/template/download")

session_payload = {
    "user_id": f"api-test-user-{RUN_LABEL}",
    "session_name": unique_label("API PYQ Session"),
    "filters": {
        "exam_id": recall("exam_id"),
        "subject_id": recall("subject_id"),
        "concept_id": recall("concept_id")
    },
    "time_limit": 30
}
run_test(
    "Create PYQ session",
    "POST",
    "pyq/session/create",
    json_body=session_payload,
    expect=(200, 201),
    store=store_pyq_session,
)

if "pyq_session_id" in state:
    run_test("PYQ current question", "GET", f"pyq/session/{recall('pyq_session_id')}/current")
    run_test("PYQ session progress", "GET", f"pyq/session/{recall('pyq_session_id')}/progress")
    run_test("PYQ pause session", "POST", f"pyq/session/{recall('pyq_session_id')}/pause")
    run_test("PYQ resume session", "POST", f"pyq/session/{recall('pyq_session_id')}/resume")
    run_test(
        "PYQ user sessions",
        "GET",
        f"pyq/sessions/user/{session_payload['user_id']}",
        params={"status": "all"}
    )

    question_ids = state.get("pyq_session_question_ids") or []
    if question_ids:
        run_test(
            "PYQ submit answer",
            "POST",
            f"pyq/session/{recall('pyq_session_id')}/submit",
            json_body={
                "question_id": question_ids[0],
                "user_answer": "Mars",
                "time_taken": 45
            },
            expect=(200, 400),
        )
        run_test(
            "PYQ navigate next",
            "POST",
            f"pyq/session/{recall('pyq_session_id')}/navigate/next",
        )
        run_test(
            "PYQ jump to first",
            "POST",
            f"pyq/session/{recall('pyq_session_id')}/jump/0",
        )


In [ ]:
summary_rows = [
    {
        "name": entry["name"],
        "method": entry["method"],
        "endpoint": entry["endpoint"],
        "status": entry["status"],
        "success": entry["success"]
    }
    for entry in test_log
]

if summary_rows:
    summary_df = pd.DataFrame(summary_rows)
    display(summary_df)
    success_count = summary_df["success"].sum()
    total = len(summary_df)
    print(f"Success rate: {success_count}/{total} ({(success_count / total) * 100:.1f}% )")
else:
    print("No tests executed yet.")
